In [ ]:
# @markdown ##**[❗]  Setup VM** {display-mode: "form"}
# @markdown Takes long time + 10GB Google Account Storage for fresh install
# Mount Google Drive to access storage
from google.colab import drive
import os

# Warning message about storage usage
warning_message = """
Warning: This process will take up to 10GB of your Google Drive storage.
This will create the 'Ubuntu22-10G' directory and download the necessary files.

Do you want to continue? (y/n)
"""

# Print the warning message
print(warning_message)

# Get user confirmation to proceed
confirm = input("Type 'y' to proceed or 'n' to cancel: ").strip().lower()

if confirm != 'y':
    print("Setup aborted. You can restart the process later.")
else:
    # Mount Google Drive to access storage
    drive.mount('/content/drive')

    # Directory for the QCOW2 image and other files
    target_dir = '/content/drive/MyDrive/Ubuntu22-10G'
    os.makedirs(target_dir, exist_ok=True)

    # Change to the target directory
    os.chdir(target_dir)

    # Install necessary packages for QEMU
    print("Installing QEMU and other required packages...")
    !export DEBIAN_FRONTEND=noninteractive && apt-get update -y && apt-get install -y wget qemu qemu-utils qemu-system-x86

    # Check if the QCOW2 image already exists
    if os.path.exists('ubuntu-22.qcow2'):
        print("ubuntu-22.qcow2 already exists. You can proceed to boot it with the second cell.")
    else:
        # Download the required Ubuntu QCOW2 image
        print("Downloading and setting up the Ubuntu QCOW2 image and other necessary files...")
        !wget --tries=50 --timeout=2 --no-hsts -O ubuntu-22.qcow2 https://cloud-images.ubuntu.com/minimal/releases/jammy/release/ubuntu-22.04-minimal-cloudimg-amd64.img

        # Download additional required files
        !wget --tries=50 --timeout=2 --no-hsts -O user-data https://raw.githubusercontent.com/rdpmakers/freeroot-KVM/refs/heads/main/user-data
        !wget --tries=50 --timeout=2 --no-hsts -O user-data.img https://github.com/rdpmakers/freeroot-KVM/raw/refs/heads/main/user-data.img

        # Resize the QCOW2 image to increase disk space (+8GB, total size will be 10GB)
        !qemu-img resize ubuntu-22.qcow2 +8G
        !touch .installed

In [ ]:
# @markdown ##**[❗]  Boot** {display-mode: "form"}
# Cell 2: "Boot it up"

# Check if the necessary files exist before booting
import os

target_dir = '/content/drive/MyDrive/Ubuntu22-10G'
qcow2_file = os.path.join(target_dir, 'ubuntu-22.qcow2')

# Warning message if the setup is incomplete
if not os.path.exists(target_dir):
    print("Warning: The QEMU directory 'Ubuntu22-10G' does not exist.")
    print("Please execute the first cell to set up the environment.")
elif not os.path.exists(qcow2_file):
    print(f"Warning: The file '{qcow2_file}' does not exist.")
    print("Please execute the first cell to download and set up the image.")
else:
    # Proceed with booting the QCOW2 image if everything is set up
    print("Booting the Ubuntu QCOW2 image with QEMU...")
    !touch .kvm-log.available
    !nohup qemu-system-x86_64 -drive file=ubuntu-22.qcow2,format=qcow2 -drive file=user-data.img,format=raw \
      -device virtio-net-pci,netdev=n0 -netdev user,id=n0,hostfwd=tcp::2222-:22,hostfwd=udp::2222-:22 \
      -smp $(nproc) -m $(awk '/MemAvailable/ {print int($2/1024/1024)"G"}' /proc/meminfo) \
      -accel tcg -cpu qemu64 -nographic &> $(date +"%Y-%m-%d_%H:%M:%S").qemu-vm.log &

In [ ]:

# @markdown ##**[❗] Setup SSH** {display-mode: "form"}
from IPython.display import clear_output
clear_output(wait=True)
!wget -O /usr/local/bin/cloudflared $(curl -s https://api.github.com/repos/cloudflare/cloudflared/releases/latest | grep -o "https://github.com/cloudflare/cloudflared/releases/download/.*/cloudflared-linux-amd64")
!chmod +x /usr/local/bin/cloudflared
!curl -L -o /tmp/wstunnel.tar.gz $(curl -s https://api.github.com/repos/erebe/wstunnel/releases/latest | grep -o "https://github.com/erebe/wstunnel/releases/download/.*/wstunnel_.*_linux_amd64.tar.gz") && tar -zxvf wstunnel.tar.gz && chmod +x wstunnel && sudo mv wstunnel /usr/local/bin/ && rm wstunnel.tar.gz
clear_output(wait=True)
!echo "ignore the error, you\'re ok"
!nohup /usr/local/bin/wstunnel server ws://localhost:80 --restrict-to localhost:2222 > wst.log &
!nohup /usr/local/bin/cloudflared tunnel --url http://localhost:80 > cfd.log &
!disown
clear_output(wait=True)
!echo "you can execute generate ssh now"

In [ ]:
# @markdown ##**[❗] Generate SSH** {display-mode: "form"}
# @markdown Install wstunnel binary on client, [download](https://github.com/erebe/wstunnel)
from IPython.display import clear_output
clear_output(wait=True)
import os
target_dir = '/content/drive/MyDrive/Ubuntu22-10G'
cfd_file = os.path.join(target_dir, 'cfd.log')
clear_output(wait=True)
# Warning message if the setup is incomplete
if not os.path.exists(cfd_file):
  !echo "Make sure you've setup the VM and have setup ssh"
else:
  !sleep 5
  !echo "execute on client: wstunnel client -L tcp://9999:localhost:2222 $(grep -oP 'https://[^\s]*trycloudflare\.com' cfd.log | sed 's/^https:/ws:/')"
  !echo "to ssh on client: ssh ubuntu@localhost -p9999"
  !echo "ssh pass: asdfqwer"

In [ ]:
# @markdown ##**[❗] Reset SSH** {display-mode: "form"}
from IPython.display import clear_output
clear_output(wait=True)
!ps aux | grep cloudflared | grep -v grep | awk '{print $2}' | xargs kill -9
!ps aux | grep wstunnel | grep -v grep | awk '{print $2}' | xargs kill -9
!rm cfd.log wst.log
clear_output(wait=True)!echo "you can setup and generate ssh again"

In [ ]:
# @markdown ##**[❗] See wstunnel log** {display-mode: "form"}
from IPython.display import clear_output
import os
target_dir = '/content/drive/MyDrive/Ubuntu22-10G'
wst_file = os.path.join(target_dir, 'wst.log')
clear_output(wait=True)
# Warning message if the setup is incomplete
if not os.path.exists(wst_file):
  !echo "Make sure you've setup the VM and have setup ssh"
else:
  !cat wst.log

In [ ]:
# @markdown ##**[❗] See cloudflared log** {display-mode: "form"}
from IPython.display import clear_output
import os
target_dir = '/content/drive/MyDrive/Ubuntu22-10G'
cfd_file = os.path.join(target_dir, 'cfd.log')
clear_output(wait=True)
# Warning message if the setup is incomplete
if not os.path.exists(cfd_file):
  !echo "Make sure you've setup the VM and have setup ssh"
else:
  !cat cfd.log

In [213]:
# @markdown ##**[❗] See VM log** {display-mode: "form"}
from IPython.display import clear_output
clear_output(wait=True)
import os
file = "2024-12-14_16:04:49.qemu-vm.log"  # @param {type:"string"}
target_dir = '/content/drive/MyDrive/Ubuntu22-10G'
qemu_file = os.path.join(target_dir, 'qemu-vm.log')
clear_output(wait=True)
# Warning message if the setup is incomplete
if not os.path.exists(qemu_file):
  !echo "Make sure you've setup the VM and have first run of the VM"
else:
  !cat {file}

cSeaBIOS (version 1.15.0-1)


iPXE (https://ipxe.org) 00:03.0 CA00 PCI2.10 PnP PMM+BFF8B4A0+BFECB4A0 CA00
                                                                               


Booting from Hard Disk...
GRUB_FORCE_PARTUUID set, attempting initrdless boot.
Linux version 5.15.0-1069-kvm (buildd@lcy02-amd64-094) (gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, GNU ld (GNU Binutils for Ubuntu) 2.38) #74-Ubuntu SMP Fri Oct 18 14:22:54 UTC 2024 (Ubuntu 5.15.0-1069.74-kvm 5.15.167)
Command line: BOOT_IMAGE=/boot/vmlinuz-5.15.0-1069-kvm root=PARTUUID=23d2b0b7-978a-4635-bdb2-1b7ea6f7c884 ro console=tty1 console=ttyS0 panic=-1
BIOS-provided physical RAM map:
BIOS-e820: [mem 0x0000000000000000-0x000000000009fbff] usable
BIOS-e820: [mem 0x000000000009fc00-0x000000000009ffff] reserved
BIOS-e820: [mem 0x00000000000f0000-0x00000000000fffff] reserved
BIOS-e820: [mem 0x0000000000100000-0x00000000bffdffff] usable
BIOS-e820: [mem 0x00000000bffe0000-0x00000000bfffffff] reserved
BIOS-e820: [mem 0x00

In [212]:
# @markdown ##**[❗] list VM log files** {display-mode: "form"}
from IPython.display import clear_output
clear_output(wait=True)
import os
target_dir = '/content/drive/MyDrive/Ubuntu22-10G'
kvm_file = os.path.join(target_dir, '.kvm-log.available')
clear_output(wait=True)
# Warning message if the setup is incomplete
if not os.path.exists(kvm_file):
  !echo "Make sure you've setup the VM and have first run of the VM"
else:
  !ls -1 *.qemu-vm.log

2024-12-14_15:42:30.qemu-vm.log
2024-12-14_16:04:49.qemu-vm.log


In [ ]:
# @markdown ##**[❗] Shutdown** {display-mode: "form"}
from IPython.display import clear_output
clear_output(wait=True)
!rm cfd.log wst.log
!apt install sshpass -y
clear_output(wait=True)
!sshpass -p asdfqwer ssh -o StrictHostKeyChecking=no -p2222 ubuntu@localhost sudo poweroff
!ps aux | grep cloudflared | grep -v grep | awk '{print $2}' | xargs kill -9
!ps aux | grep wstunnel | grep -v grep | awk '{print $2}' | xargs kill -9
clear_output(wait=True)
!echo "your server has shutdowned properly"